# Compañia movil

## Inicialización


In [1]:
#importamos las librerias necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Cargar datos

In [2]:
#cargar el archivo
try:
    user = pd.read_csv('/datasets/users_behavior.csv')
except:
    user = pd.read_csv(r'\datasets\users_behavior.csv')

In [3]:
user.info()
user.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Preparar datos

In [4]:
#verificar si el cambio de float a entero no se eliminara los decimales
np.array_equal(user['calls'], user['calls'].astype('int'))

True

In [5]:
#verificar si el cambio de float a entero no se eliminara los decimales
np.array_equal(user['messages'], user['messages'].astype('int'))

True

In [6]:
#cambiar a entero
user['calls']=user['calls'].astype('int')

In [7]:
#cambiar a entero
user['messages']=user['messages'].astype('int')

In [8]:
#informacion de los datos
user.info()
user.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0


Se convirtió a tipo de dato entero los elementos de las columnas *'calls'* y *'messages'* previa comprobación para no afectar su calidad.

## Segmenación de datos

Se dividira el DataSet en conjunto de entrenamiento, validación y de prueba(60%, 20%, 20% respectivamente).

In [9]:
#identificar el feature y target
features = user.drop(['is_ultra'], axis=1)
target = user['is_ultra']

In [10]:
#segmentamos los datos, utilizando el 20% para el testeo de datos y el resto para el entrenamiento y validación en los 
#diferentes modelos 
train_valid, test, train_valid_target, test_target  = train_test_split(features, target, test_size = 0.20,
                                                                     random_state = 12345)


In [11]:
#segmentamos los datos, utilizando el 60% para el entrenamiento de datos y el resto para validación en los 
#diferentes modelos 
train, valid, train_target, valid_target  = train_test_split(train_valid, train_valid_target, test_size = 0.25,
                                                                     random_state = 12345)


Se utilizará lo expuesto para concluir la exactitud de cada modelo.

## Modelos de estudio (conjunto de entrenamiento y validación)

### Arboles de decisión

In [12]:
#variables inciales
best_model = None
best_result = 0
best_depth = 0

In [13]:
#Recorremos la profundidad en un rango de numeros en el modelo de Arbol de decisión
#Para encontrar su exactitud
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model.fit(train, train_target)
    prediction_valid = model.predict(valid)
    
    result = accuracy_score(valid_target, prediction_valid)
    #print('max_depth', depth, ':', result)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print('Exactitud del mejor modelo en el conjunto de validación donde max_depth={} : {}'.format(best_depth, best_result))

Exactitud del mejor modelo en el conjunto de validación donde max_depth=7 : 0.7744945567651633


Encontramos una mayor exactitud del modelo de 0.77 cuando *depth* es de 7 para el conjunto de validación.

### Bosque aleatorio

In [14]:
#valores iniciales
best_score = 0
best_est = 0

In [15]:
#Recorremos el n_estimators en un rango de numeros en el modelo de Bosque aleatorio
#Para encontrar su exactitud
for est in range(1, 45):
    model = RandomForestClassifier(random_state = 54321, n_estimators = est)
    model.fit(train, train_target)
    score = model.score(valid, valid_target)
    
    if score > best_score:
        best_score = score
        best_est = est
        
print('La exactitud del mejor modelo en el conjunto de validación donde n_estimators={} : {}'.format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación donde n_estimators=35 : 0.7978227060653188


Se encuentra una exactitud de 0.79 en el conjunto de validación utilizando el numero de estimacion = 35.

### Regresión Logística

In [16]:
#Modelo de rgrsion logistica ingresando los parametros 
model = LogisticRegression(random_state = 12344, solver = 'liblinear')
model.fit(train, train_target)

#accuracy en el conjunto de validación
score_valid = model.score(valid, valid_target)
print('Exactitud en el conjunto de validación: ', score_valid)

Exactitud en el conjunto de validación:  0.7293934681181959


Se encuentra una exactitud con el modelo de Regresion Logistica de 0.72.

***Bosque aleatorio*** nos brinda una mayor exactitud con respecto a los otros dos modelos con una exactitud de 0.79 cuando n_estimators = 35.

## Modelos de estudio en el conjunto de Prueba

Se escoje el siguiente modelo para analizar el comportamiento en el conjunto de prueba.

### Bosque aleatorio

In [17]:
model = RandomForestClassifier(random_state = 44444, n_estimators = 45)
model.fit(valid, valid_target)
score = model.score(test, test_target)
print('Bosque aleatorio con exactitud en todo el conjunto de prueba es de: ', score)

Bosque aleatorio con exactitud en todo el conjunto de prueba es de:  0.7822706065318819


La exactitud del conjunto de prueba es de 0.78.

## Prueba de cordura

In [18]:
sanity_test = user['is_ultra'].value_counts()/user.shape[0]
sanity_test

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

El 69% de los planes no es Ultra.

## Conclusiones

1. Al realizarse la prueba de cordura se observa que se tiene 69% de usuarios Smart, comparado con el modelo de la Regresión logística no se veria una mejora que si eventualmente se tiene un modelo alterno de columna *is_ultra* de "0".


2. El modelo de Clasificación de Bosque aleatorio evaluado en el conjunto de validación brinda una exactitud de 0,79. 
   
   Por otro lado, la exactitud en el conjunto de prueba es de 0,78. Esto menciona que un aproximado de 8 de cada 10 elementos evaluados es acertado.


3. En los modelos expuestos se utilizó hiperparametros probados de tal forma que sean el necesario y no en exceso para no recargar la maquina y que no realice cambios relevantes en su exactitud.